Note: This is only a **temporary** notebook that is used for development purposes. 

In [198]:
%load_ext autoreload
%autoreload 2


import os
import sys
import numpy as np
import pandas as pd

from numpy.matlib import repmat

from config import config
import fit_ac_fc as f

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [200]:
ab = f.load_abundance_thresholds(os.path.join(config.A549_test, \
        "A549_abundance_thresholds.txt"))
tps = f.load_timepoint_counts(os.path.join(config.A549_test, \
        "A549_timepoint_counts.txt"))

In [202]:
_tps, names = f.convert_timepoint_counts(tps)
_abundance = f.convert_abundance_thresholds(ab)

In [204]:
# For testing only

_tps = _tps.iloc[:100,:]
_abundance = _abundance.iloc[:100,:]

# fit_ac_fc function

In [205]:
def create_mask(counts, abundance, n_good=2): 
    pass_mask = pd.DataFrame(counts.values > abundance.values, columns=counts.columns)
    mask = pass_mask.groupby(level=0, axis=1).apply(lambda x: x.sum(axis=1) >= n_good)
    
    return mask 

def masked_cov(x,y): 
    term1 = np.ma.mean(np.ma.multiply(x,y), axis=1)
    term2 = np.multiply(np.ma.mean(x, axis=1), np.ma.mean(y, axis=1))
    
    return term1 - term2

In [206]:
def fit_ac_fc_np(counts, abundance, times, n_good=2):
    #TODO: parse times
    
    mx_s, mt_s, v_s, f_s = [], [], [], []
    for count, ab in zip(counts, abundance): 
        mask = create_mask(count, ab, n_good=n_good)
        
        count_ma = np.ma.masked_array(count, mask=~mask)
        time_ma  = np.ma.masked_array(repmat(times, count.shape[0], 1), mask=~mask)
        
        mx = count_ma.mean(axis=1)
        mt = time_ma.mean(axis=1)
        v  = time_ma.var(axis=1)
        f = masked_cov(count_ma, time_ma)
        
        mx_s.append(mx)
        mt_s.append(mt)
        v_s.append(v_s)
        f_s.append(f)
        
    f_sum = np.zeros(counts[0].shape[0],)
    v_sum = np.zeros(counts[0].shape[0],)
    for f, v in zip(f_s, v_s): 
        f_sum = np.add(f_sum, f)
        v_sum = np.add(v_sum, v)
        
    fc = np.divide(f_sum, v_sum)
    
    ac_s = []
    for mx, mt in zip(mx, mt): 
        ac = mx - np.multiply(fc, mt)
        ac_s.append(ac)

In [207]:
%time fit_ac_fc_np([_tps[1], _tps[2]], [_abundance[1], _abundance[2]], np.array([3, 14, 21, 28]))

KeyboardInterrupt: 

In [209]:
a = np.arange(0,10)

In [212]:
np.power(2,a)

array([  1,   2,   4,   8,  16,  32,  64, 128, 256, 512])

In [87]:
np.__version__

'1.12.1'

In [96]:
_data = _tps[1].values
_ab = _abundance[1].values

In [97]:
_data > _ab

array([[ True, False, False, False],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       ..., 
       [ True,  True, False,  True],
       [ True,  True,  True,  True],
       [False, False, False, False]], dtype=bool)

In [165]:
data_masked = np.ma.masked_array(_data, mask=_data < _ab)
times_masked = np.ma.masked_array(repmat(times, data.shape[0], 1), mask=data_masked.mask)

In [166]:
mx = data_masked.mean(axis=1)
mt = times_masked.mean(axis=1)
v  = times_masked.var(axis=1)
f = masked_cov(data_masked, times_masked)

In [66]:
def fit_ac_fc(counts, abundance, n_good=2): 
    #Consider doing this in pure numpy 
    
    assert (counts.columns == abundance.columns).all()
    
    return create_mask(counts, abundance, n_good=n_good)

    # aggregate mask for "allbad"
    
    
    #return pass_mask